In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

import wget
from pathlib import Path

from matplotlib import pyplot as plt

import traceback
import warnings 

warnings.filterwarnings('ignore')

np.random.seed(0)

%matplotlib inline

pd.set_option('display.min_rows', 21)
pd.set_option('display.max_colwidth', None)

In [2]:
root="/home/paperspace/amex-default-prediction/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
scols = [col for col in headers if col[0] == 'S']
dcols = [col for col in headers if col[0] == 'D']
rcols = [col for col in headers if col[0] == 'R']
bcols = [col for col in headers if col[0] == 'B']
pcols = [col for col in headers if col[0] == 'P']
labels =pd.read_csv(root+"train_labels.csv")

In [3]:
trains = []
for r, d, f in os.walk(root):
    f.sort()
    for file in f:
#        if "traina" in file or "trainb" in file:
        if 'pkl' not in file and 'csv' not in file and ("xa" in file or "xb" in file or file.startswith("xc")):
            trains.append(os.path.join(root, file))

dates=pd.DataFrame(pd.date_range('2018-04-01','2019-04-30', freq='MS').tolist(), columns=['S_2_dt'])
dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")

In [18]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

traindf = pd.read_csv(root+"/tdata")
tdf = traindf.groupby('customer_ID')['S_2'].agg(['min', 'max', 'count']).reset_index()
#tdf = tdf.merge(labels, on="customer_ID", how="inner")
#for i in range(13):
#    print(i, tdf[tdf['S_2']==i].count())


In [ ]:
tdf.rename(columns={"min": "mindate", "max": "maxdate"}, inplace=True)

tdf['maxdate'] = pd.to_datetime(tdf['maxdate'])
tdf['mindate'] = pd.to_datetime(tdf['mindate'])

#tdf['diffcount'] = tdf.apply(lambda x: ((x['maxdate'] - x['mindate'])/np.timedelta64(1, 'M')))

tdf['mondiff'] = (tdf.maxdate.dt.strftime("%Y%m").astype(int) - tdf.mindate.dt.strftime("%Y%m").astype(int) - 87)
tdf['mondiff'] = tdf['mondiff'].apply(lambda x: x+88 if x< 0 else x)
#tdf[tdf['count'] == tdf['mondiff']]

In [ ]:
tdf[tdf['customer_ID']=='002b66a636237d16b2344db6c39210bae5f091c9caf8479e6066101b4a197ce0']

In [ ]:
traindfs = []

for k in range(13):
    traindfs.append(pd.DataFrame(columns=headers))
k=0
for i in range(len(trains)):
    print(trains[i])
    traindf = pd.read_csv(trains[i], names=headers, header=None)
    for j in range(1, 14):
        mask = traindf.customer_ID.isin(tdf[tdf['mondiff'] == j].customer_ID)
        traindfs[j-1] = pd.concat([traindfs[j-1], traindf[mask]], ignore_index=True)
        if traindfs[j-1]['customer_ID'].count() >= 260000:
#            traindfs[j-1] = traindfs[j-1].merge(labels, on="customer_ID", how="inner")
            traindfs[j-1].iloc[:260000,].to_csv("x" + str(j-1) + "-" + str(k) + ".csv", index=False)
            traindfs[j-1] = traindfs[j-1].iloc[260000:,]
            print("x" + str(j-1) + "-" + str(k) + ".csv")
#            traindfs[j-1].drop(columns=['target'], inplace=True)
            k+=1            

for i in range(len(traindfs)-1):
#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].to_csv("x" + str(i) + ".csv", index=False)
    print("x" + str(i) + ".csv")

for i in range(12, len(traindfs)):
#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].to_csv("x" + str(i) + "-" + str(k) + ".csv", index=False)
    print("x" + str(i) + "-" + str(k) + ".csv")


In [ ]:
mdf = pd.DataFrame()
for m in range(1):
    print("x12-" + str(m) + ".csv")
    mdf = pd.concat([mdf, pd.read_csv("x12-"+str(m)+".csv")], ignore_index=True)
mdf['tag'] = 12
mdf['S_2'] = pd.to_datetime(mdf['S_2'])
mdf['S_2'] = mdf['S_2'] + pd.offsets.DateOffset(years=1)
mdf['S_2'] = mdf['S_2'] + pd.offsets.DateOffset(months=1)
mdf['day_of_month'] = mdf['S_2'].dt.strftime("%d")
mdf['day_of_week'] = mdf['S_2'].dt.strftime("%u")
mdf['day_of_year'] = mdf['S_2'].dt.strftime("%j")
mdf['week_of_year'] = mdf['S_2'].dt.isocalendar().week
mdf['yearmonth'] = mdf['S_2'].dt.strftime("%Y%m")

#mdf = mdf.merge(labels, on="customer_ID", how="inner")

In [ ]:
traindfs = []

print("Loading data...", end=" ")
for j in range(12):
    print("x" + str(j), end=" ")
    traindfs.append(pd.read_csv("x" + str(j) + ".csv"))
#    traindfs[j]['tag'] = j
    traindfs[j]['S_2'] = pd.to_datetime(traindfs[j]['S_2'])
    traindfs[j]['day_of_month'] = traindfs[j]['S_2'].dt.strftime("%d")
    traindfs[j]['day_of_week'] = traindfs[j]['S_2'].dt.strftime("%u")
    traindfs[j]['day_of_year'] = traindfs[j]['S_2'].dt.strftime("%j")
    traindfs[j]['week_of_year'] = traindfs[j]['S_2'].dt.isocalendar().week
    traindfs[j]['yearmonth'] = traindfs[j]['S_2'].dt.strftime("%Y%m")

i=0
print("")
print("Transforming data...", end=" ")
for j in range(12):
    print("x" + str(j), end=" ")

    maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
    mindate = maxdate + pd.offsets.DateOffset(months= -1-j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=(dates['seq']+1)

    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonthorig'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i]['seq'] = traindfs[i].groupby(['customer_ID']).cumcount().add(1)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = traindfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2018 and x['mondiff'] > j else x['mondiff'], axis=1)
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <= 0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
#    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    traindfs[i]['yearmonth'] = traindfs[i].apply(lambda x: x['yearmonth_y'] if pd.isnull(x['yearmonth_x']) else x['yearmonth['S_2_'], axis=1)
#    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
#    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
#    traindfs[i].drop(columns=['S_2_dt', 'origmissrcnt', 'totmissrcnt', 'seq_x', 'seq_y'], inplace=True)
    traindfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
    traindfs[i] = traindfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    traindfs[i]['yearmonthorig'].fillna(0, inplace=True)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']].fillna(-1, inplace=True)
#    traindfs[i]['mondiff'].fillna(0, inplace=True)
    traindfs[i].drop(columns=['mondiff'], inplace=True)

#    traindfs[i]['tag'] = j
#    traindfs[i] = pd.concat([traindfs[i], pd.merge(mdf, dates['yearmonth'], on=['yearmonth'],how='inner')], ignore_index=True)

#    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
#    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
#    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
#    traindfs[i]['week_of_year'] = traindfs[i]['S_2'].dt.isocalendar().week
#    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i].drop(columns=['S_2'], inplace=True)
##    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")

##    cols=traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
##    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##   traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i][cols] = traindfs[i].groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i].drop(columns=['target'], inplace=True)
##    for k in range(i+1, 12):
##        traindfs[i] = pd.concat([traindfs[i], traindfs[k]], ignore_index=True)

##    traindfs[i].drop(columns=['S_2'], inplace=True)
##    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")

##    cols=traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
##    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
##    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

##    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i][cols] = traindfs[i].groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

##    traindfs[i].drop(traindfs[i][traindfs[i]['tag']!=j].index, axis=0, inplace=True)
##    traindfs[i].drop(columns=['tag'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
#    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    traindfs[i].fillna(0, inplace=True)

#    cols=traindfs[i].columns
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][[column for column in cols if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].groupby("target")[col].transform(lambda x: x.fillna(x.mean()), inplace=True)
#        traindfs[i][col].fillna(-999, inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i].groupby("target")[col].transform(lambda x: x.fillna(x.mean()), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

#    for column in [col for col in traindfs[0].columns if col[len(col)-6:] in ['201904', '201903']]:
#        traindfs[0].drop(columns=[column], inplace=True)

    traindfs[i].to_csv("x" + str(j) +"13dts.csv", index=False)
    i+=1

traindfs[0]

In [ ]:
traindfs[0] #[(traindfs[3]['origmissrcnt201904'] >0 ) | (traindfs[3]['origmissrcnt201903']>0) | (traindfs[3]['origmissrcnt201902']>0) | (traindfs[3]['origmissrcnt201901']>0)][['origmissrcnt201901', 'origmissrcnt201902', 'origmissrcnt201903', 'origmissrcnt201904']]
#traindfs[0].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
traindfs[2]

In [ ]:
#traindf=pd.read_csv("mdfdtcnt.csv")
traindf=pd.DataFrame()
for j in range(2):
    print(j, end=" ")
    traindf = pd.concat([traindf, traindfs[j]], ignore_index=True)
#    cols=traindf[traindf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    traindf[cols] = traindf.groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindf[cols] = traindf.groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindf[cols] = traindf.groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindf[cols] = traindf.groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindf[cols] = traindf.groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindf[cols] = traindf.groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindf[cols] = traindf.groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

cols=traindf[traindf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

traindf #[['target', 'B_1201804', 'B_10201804', 'B_11201804', 'B_12201804', 'B_13201804', 'B_14201804', 'B_15201804', 'B_16201804', 'B_1201805', 'B_10201805', 'B_11201805', 'B_12201805', 'B_13201805', 'B_14201805', 'B_15201805', 'B_16201805']]

traindf#.groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

In [ ]:
traindf[cols] = traindf.groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))
cols=traindf[traindf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
traindf[['target', 'B_1201806', 'B_10201806', 'B_11201806', 'B_12201806', 'B_13201806', 'B_14201806', 'B_15201806', 'B_16201806', 'B_1201805', 'B_10201805', 'B_11201805', 'B_12201805', 'B_13201805', 'B_14201805', 'B_15201805', 'B_16201805']]


In [ ]:
## grpcols = [col for col in traindfs[i].columns if (col == 'target') or (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]
#grpcols
#traindfs[i].groupby(['yearmonth', 'target'])[col].transform(lambda x: x.fillna(x.mean()))
#yearmonth 	day_of_month 	day_of_week 	day_of_year 	target
#traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[col].agg(['mean'])
#traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[col].agg(['mean'])
#traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[col].agg(['mean'])
#traindfs[i].groupby(['target', 'day_of_week'])[col].agg(['mean'])

cols=mdf[mdf.select_dtypes(include=['int64', 'float64']).columns].isna().any().index
cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#mdf[cols] = mdf.groupby(['day_of_year', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))
#mdf[cols] = mdf.groupby(['day_of_year'])[cols].transform(lambda x: x.fillna(x.mean()))
#mdf.groupby(['target', 'day_of_year', 'day_of_week']).transform(lambda x: x.fillna(x.mean()))
traindfs[1]#[['customer_ID', 'B_39201904', 'B_42201904', 'target']]

In [ ]:
traindfs = []
i=0
j=12
for k in range(20):
    print("x" + str(j) + "-" + str(k), end=" ")
    traindfs.append(pd.read_csv("x" + str(j) + "-" + str(k) + ".csv"))

    maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
    mindate = maxdate + pd.offsets.DateOffset(months= -1-j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=(dates['seq']+1)

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonthorig'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i]['seq'] = traindfs[i].groupby(['customer_ID']).cumcount().add(1)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = traindfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2018 and x['mondiff'] > j else x['mondiff'], axis=1)
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <= 0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
#    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    traindfs[i]['yearmonth'] = traindfs[i].apply(lambda x: x['yearmonth_y'] if pd.isnull(x['yearmonth_x']) else x['yearmonth['S_2_'], axis=1)
#    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
#    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
#    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
#    traindfs[i] = traindfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    traindfs[i]['yearmonthorig'].fillna(0, inplace=True)
#    traindfs[i]['mondiff'].fillna(0, inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i]['week_of_year'] = traindfs[i]['S_2'].dt.isocalendar().week
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)

#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")

#    cols=traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    traindfs[i][cols] = traindfs[i].groupby(['target', 'yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    traindfs[i][cols] = traindfs[i].groupby(['target', 'day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#    traindfs[i][cols] = traindfs[i].groupby(['target'])[cols].transform(lambda x: x.fillna(x.mean()))

#    traindfs[i].drop(columns=['target'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
#    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    traindfs[i].fillna(0, inplace=True)

#    cols=traindfs[i].columns
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][[column for column in cols if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(-999, inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

#    for column in [col for col in traindfs[0].columns if col[len(col)-6:] in ['201904', '201903']]:
#        traindfs[0].drop(columns=[column], inplace=True)

    traindfs[i].to_csv("x" + str(j) + "-" + str(k) +"dtcnt.csv", index=False)
    i+=1

traindfs[0]

In [ ]:
traindfs[0].iloc[traindfs[0].notna().index] #[traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index

In [ ]:
traindfs = []

for j in range(len(trains)):
    i = j#-18
    print(trains[j])
    traindfs.append(pd.read_csv(trains[j], names=headers, header=None))

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt'], inplace=True)

    traindfs[i]['mondiff'].fillna(0, inplace=True)
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = traindfs[i].groupby('customer_ID')[col].transform(lambda grp: grp.fillna(np.mean(grp)))
#    for col in traindfs[i].columns[traindfs[i].isna().any()]:
#        if traindfs[i][col].dtype in ['float64', 'int64']:
#            traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][[column for column in traindfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

    traindfs[i].to_csv(trains[i]+"nona.csv", index=False, header=False)

traindfs[0][traindfs[0]['customer_ID']==0].to_csv(root+"/trains/headers.csv", index=False)
traindfs[0]

In [ ]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

traindf = pd.read_csv(root+"/tdata")


In [ ]:
#traindf = traindf[traindf['target']==1]
#for col in [col for col in traindf.columns if (col[len(col)-6:]=='201804')]:
#    traindf = traindf[~traindf[col].isnull()]

#traindf=traindf[~traindf['B_12201804'].isnull()]
#traindf['B_42201804'].unique()

#(traindf.groupby('customer_ID')['S_2'].agg(["min", "max"]).reset_index())['min'].max(), (traindf.groupby('customer_ID')['S_2'].agg(["min", "max"]).reset_index())['max'].min()

tdf = traindf.groupby('customer_ID')['S_2'].count().reset_index()
tdf = tdf.merge(labels, on="customer_ID", how="inner")
for i in range(1, 13):
    print(i, tdf[tdf['S_2']==i].count())

#traindf.groupby('customer_ID')['S_2'].count().reset_index()

In [13]:
traindf=pd.DataFrame()

for i in range(12):
    print("x" + str(i), end=" ")
    traindf = pd.concat([traindf, pd.read_csv("x" + str(i) + "13dts.csv")], ignore_index=True)

[traindf[col].astype('category') for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]
#    traindf[col] = traindf[col].astype('category')
#print("updated categories...")

traindf

x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 

,customer_ID,B_1201904,B_10201904,B_11201904,B_12201904,B_13201904,B_14201904,B_15201904,B_16201904,B_17201904,...,day_of_week201805,day_of_year201805,dncnt201805,origmissrcnt201805,pncnt201805,rncnt201805,sncnt201805,totmissrcnt201805,week_of_year201805,yearmonthorig201805
0,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d5ec50ef6aa6487cc14,0.003712,0.028748,0.003663,0.010627,0.006496,0.001786,0.009511,0.002692,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000eee12732c9c7b790870c556bc33339d6d17725cbeb4a7da6b2a5d2c8386aa,0.079262,0.296491,0.052629,0.013833,0.046850,0.023636,0.003698,0.003267,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0015d621bacbfd59cc1479aea405645ecb480f6ee779e06d66ce55da3bd397fd,0.048989,0.035238,0.031642,0.013585,0.024790,0.023208,0.024831,0.004788,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00163cf7b8cc9cb2781ac05331d83c3ae7720f4860ee8aa2a3f0fb70b13f2f25,1.317920,0.126040,1.337500,0.009768,0.078476,0.330606,0.002271,1.004898,0.959150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,004bf15cf65c01b2173e53c5284933d41650e69d977ebd737e0c8cda9af483d8,0.046566,0.238808,0.026537,0.001177,999.000000,0.011219,0.007862,0.007938,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,005750fedff03417405a5ba8485c28bafb87c4149668b9a1c3d1763baf410d0e,0.017442,0.002560,0.009883,0.012196,999.000000,0.006943,0.005328,0.001940,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,005a7f765eee4afa0455c5a3662ab8bc352b49cabac0ed5d0765a67881920705,1.317488,0.009560,1.379268,0.012404,0.031268,0.441518,0.007766,1.004258,0.981506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0066a59c8b96adec9e847a64d83d481f8b67eb6b96eac02d52cc021181b9c147,0.089652,0.001285,0.068761,0.035229,0.078587,0.073792,0.082613,0.004725,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,007640e6d0fb8423be50a40c337108db80471dae973fc840ee98cdbbbed10038,0.031857,0.293505,0.016359,0.013324,0.008357,0.013439,0.008173,0.088800,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,007bbdf03ea7c57f88352c8b01fb7da136e759f7b8a063fabc7187e4dc3f07cb,0.014073,0.245036,0.001191,0.059974,0.006510,0.017528,0.017581,0.003313,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
traindf=pd.DataFrame()

for i in range(4):
    print("x12-" + str(i), end=" ")
    traindf = pd.concat([traindf, pd.read_csv("x12-" + str(i) + "dtcnt.csv")], ignore_index=True)

[traindf[col].astype('category') for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]
#    traindf[col] = traindf[col].astype('category')
#print("updated categories...")

traindf

In [26]:
def preprocess(X, impute_numericals, impute_categories, encode_categories, imputation_type='simple'):
    X.drop(columns=['customer_ID'], inplace=True)
    print("dropped customer_ID.", end=" ")

    if impute_categories:
        cols = [col for col in X.columns if col[:-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]
#        cols=list(X[X[cols].select_dtypes(include=['object']).columns].isna().any().index)
        imp = SimpleImputer(missing_values=np.nan, strategy='constant')
        imp.fit(X[cols])
        X[cols]=imp.transform(X[cols])
        X[cols] = X[cols].astype("str")
        print("imputed categoricals.", end=" ")

    if encode_categories:
        cols = [col for col in X.columns if col[:-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]
        X[cols] = X[cols].astype("str")
        enc = preprocessing.OrdinalEncoder()
        enc.fit(X[cols])
        X[cols]=enc.transform(X[cols])
        print("encoded categories.", end=" ")

    if impute_numericals:
        cols=list(X.select_dtypes(include=['int64', 'float64']).columns[X.select_dtypes(include=['int64', 'float64']).isna().any()])
        #cols=list(cols - set(X.select_dtypes(include=['int64', 'float64']).columns[X.select_dtypes(include=['int64', 'float64']).notna().all()]))
        if imputation_type == 'knn':
            imp = KNNImputer()
        else:
            imp = SimpleImputer()
        X[cols] = imp.fit_transform(X[cols])
        print("imputed numericals.", end=" ")

    return X

def split(X):
    y=X['target']
    X.drop(columns=['target'], inplace=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1)
    print("train/test done.", end=" ")

    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,test_size=0.1)
    print("train/validation done.", end=" ")

    return X_train, X_valid, y_train, y_valid, X_test, y_test


In [5]:
class Gini(Metric):
    def __init__(self):
        self._name = "gini"
        self._maximize = True

    def __call__(self, y_true, y_score):
        auc = roc_auc_score(y_true, y_score[:, 1])
        return max(2*auc - 1, 0.)

In [27]:
X = preprocess(traindf.loc[:, ~traindf.isnull().all()].fillna(999), False, False, True) #imputation_type- simple, iterative, knn

X_train, X_valid, y_train, y_valid, X_test, y_test = split(X)

X_train

dropped customer_ID. encoded categories. train/test done. train/validation done. 

,B_1201904,B_10201904,B_11201904,B_12201904,B_13201904,B_14201904,B_15201904,B_16201904,B_17201904,B_18201904,...,day_of_week201805,day_of_year201805,dncnt201805,origmissrcnt201805,pncnt201805,rncnt201805,sncnt201805,totmissrcnt201805,week_of_year201805,yearmonthorig201805
2953,0.054351,0.037771,0.034554,0.007454,999.000000,0.019427,0.001054,0.009572,999.000000,0.650195,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
48248,0.026596,0.090576,0.014503,0.036859,0.050742,0.016447,0.008184,0.335571,999.000000,0.646651,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
26151,0.053441,0.062222,0.025929,0.005652,0.020098,0.012385,0.003988,0.256991,0.840382,0.484540,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
4858,0.001508,0.063845,0.004952,0.012746,0.129988,0.006423,0.003403,0.834854,999.000000,0.594235,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
47361,0.045725,0.281129,0.028531,0.028004,0.033483,0.026045,0.002953,0.590917,0.277105,0.240763,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
10492,0.083546,0.218438,0.052475,0.011433,999.000000,0.024031,0.008541,0.009751,999.000000,0.690751,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
19572,0.038576,0.121773,0.022628,0.018440,0.024563,0.023715,0.004823,0.001917,999.000000,0.691943,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2021,0.052928,0.297606,0.035575,0.014302,0.009296,0.018304,0.009265,0.008004,999.000000,1.002422,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
50447,0.008917,0.031164,0.003287,0.009945,0.002365,0.001396,0.001941,0.000170,999.000000,0.646837,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
29905,0.047303,0.248917,0.039323,0.210473,0.236443,0.172944,0.002674,0.084975,0.961948,0.542892,...,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


In [32]:
#X.to_csv("X011.csv", index=False)
traindf

,customer_ID,B_1201904,B_10201904,B_11201904,B_12201904,B_13201904,B_14201904,B_15201904,B_16201904,B_17201904,...,day_of_week201805,day_of_year201805,dncnt201805,origmissrcnt201805,pncnt201805,rncnt201805,sncnt201805,totmissrcnt201805,week_of_year201805,yearmonthorig201805
0,0009bd8871169b8d76a445e1f52079a6edc0b6f0bf021d5ec50ef6aa6487cc14,0.003712,0.028748,0.003663,0.010627,0.006496,0.001786,0.009511,0.002692,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000eee12732c9c7b790870c556bc33339d6d17725cbeb4a7da6b2a5d2c8386aa,0.079262,0.296491,0.052629,0.013833,0.046850,0.023636,0.003698,0.003267,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0015d621bacbfd59cc1479aea405645ecb480f6ee779e06d66ce55da3bd397fd,0.048989,0.035238,0.031642,0.013585,0.024790,0.023208,0.024831,0.004788,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00163cf7b8cc9cb2781ac05331d83c3ae7720f4860ee8aa2a3f0fb70b13f2f25,1.317920,0.126040,1.337500,0.009768,0.078476,0.330606,0.002271,1.004898,0.959150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,004bf15cf65c01b2173e53c5284933d41650e69d977ebd737e0c8cda9af483d8,0.046566,0.238808,0.026537,0.001177,999.000000,0.011219,0.007862,0.007938,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,005750fedff03417405a5ba8485c28bafb87c4149668b9a1c3d1763baf410d0e,0.017442,0.002560,0.009883,0.012196,999.000000,0.006943,0.005328,0.001940,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,005a7f765eee4afa0455c5a3662ab8bc352b49cabac0ed5d0765a67881920705,1.317488,0.009560,1.379268,0.012404,0.031268,0.441518,0.007766,1.004258,0.981506,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0066a59c8b96adec9e847a64d83d481f8b67eb6b96eac02d52cc021181b9c147,0.089652,0.001285,0.068761,0.035229,0.078587,0.073792,0.082613,0.004725,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,007640e6d0fb8423be50a40c337108db80471dae973fc840ee98cdbbbed10038,0.031857,0.293505,0.016359,0.013324,0.008357,0.013439,0.008173,0.088800,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,007bbdf03ea7c57f88352c8b01fb7da136e759f7b8a063fabc7187e4dc3f07cb,0.014073,0.245036,0.001191,0.059974,0.006510,0.017528,0.017581,0.003313,999.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
clf = TabNetClassifier() #TabNetRegressor()
clf.fit(
    X_train.values, y_train.values,
    eval_set=[(X_valid.values, y_valid.values)],
    eval_metric=['auc', 'logloss', 'balanced_accuracy', 'gini']
)

Device used : cuda
epoch 0  | loss: 0.69589 | val_0_auc: 0.56476 | val_0_logloss: 1.22856 | val_0_balanced_accuracy: 0.50858 | val_0_gini: 0.12952 |  0:00:06s
epoch 1  | loss: 0.58753 | val_0_auc: 0.72524 | val_0_logloss: 0.6656  | val_0_balanced_accuracy: 0.63497 | val_0_gini: 0.45047 |  0:00:12s
epoch 2  | loss: 0.5824  | val_0_auc: 0.74447 | val_0_logloss: 0.64044 | val_0_balanced_accuracy: 0.67343 | val_0_gini: 0.48894 |  0:00:19s
epoch 3  | loss: 0.57494 | val_0_auc: 0.7209  | val_0_logloss: 0.63443 | val_0_balanced_accuracy: 0.64259 | val_0_gini: 0.4418  |  0:00:25s
epoch 4  | loss: 0.56223 | val_0_auc: 0.7336  | val_0_logloss: 0.62028 | val_0_balanced_accuracy: 0.66119 | val_0_gini: 0.46719 |  0:00:31s
epoch 5  | loss: 0.54648 | val_0_auc: 0.75834 | val_0_logloss: 0.58661 | val_0_balanced_accuracy: 0.62188 | val_0_gini: 0.51668 |  0:00:38s
epoch 6  | loss: 0.5369  | val_0_auc: 0.76818 | val_0_logloss: 0.56591 | val_0_balanced_accuracy: 0.68675 | val_0_gini: 0.53636 |  0:00:44s
e

In [29]:
preds = clf.predict(X_test.values)

probs = clf.predict_proba(X_test.values)
test_auc = roc_auc_score(y_score=probs[:,1], y_true=y_test)

preds_valid = clf.predict_proba(X_train.values)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_train)

print(f"BEST VALID SCORE : {clf.best_cost}")
print(f" TEST SCORE : {test_auc}")
print(f"TRAIN SCORE : {valid_auc}")


BEST VALID SCORE : 0.7913958625862723
 TEST SCORE : 0.9039248796684666
TRAIN SCORE : 0.9048411757845676


In [30]:
testdfs=[]
print("Loading base data...")
for i in range(1):
    print("t12-" + str(i), end=" ")
    testdfs.append(pd.read_csv("t12-" + str(i) + "dtcnt.csv"))

print("")
print("Loading base data...")
for i in range(1):
    print("t12-" + str(i), end=" ")
    testdfs[i] = preprocess(testdfs[i].fillna(999), False, False, True, 'simple')


Loading base data...
t12-0 
Loading base data...
t12-0 dropped customer_ID. encoded categories. 

In [31]:
preds_valid = clf.predict_proba(testdfs[0][list(set(traindf.columns)-set(['customer_ID', 'target']))].values)

pd.DataFrame(preds_valid)

,0,1
0,0.983497,0.016504
1,0.983496,0.016504
2,0.983407,0.016593
3,0.983338,0.016662
4,0.983497,0.016503
5,0.983489,0.016511
6,0.983346,0.016654
7,0.983441,0.016559
8,0.983496,0.016504
9,0.983490,0.016510


In [39]:
tpreds=[]
for i in range(1):
    tpreds.append(pd.read_csv(root+"/test/" + "t12-" + str(i) + ".preds"))
tpreds[0]=pd.concat([tpreds[0], pd.DataFrame(preds_valid)], axis='columns')
tpreds[0]

In [47]:
tpreds[0]['pred1']=round(tpreds[0]['Label'])
tpreds[0]['pred2']=round(tpreds[0][0])
tpreds[0][tpreds[0]['pred2']==1].count()

customer_ID    53
Label          53
0              53
1              53
pred1          53
pred2          53
dtype: int64

In [ ]:
#pd.DataFrame(imp.transform(traindf[cols]))
len(cols), cols

#list(
#traindf[
#traindf[traindf.columns].select_dtypes(include=['int64', 'float64']).columns#].notna().all().index
     #.isna().all().index)
#len(traindf.select_dtypes(include=['int64', 'float64']).columns[traindf.select_dtypes(include=['int64', 'float64']).notna().any()])

In [ ]:
#load_config("/amex/dtclassconfig.pkl")
dt = create_model('dt', fold=6, verbose=True)
print("created")
save_model(dt, "/amex/models/dt2reg")

In [ ]:
save_model(dt, "/amex/models/dt1Lclass")

In [ ]:
dt = tune_model(dt, fold=6, n_iter=10, choose_better=True)
print("tuned")
save_model(dt, "e:/rebid/amex/models/dt1Lclasstuned")

In [ ]:
dt = finalize_model(dt)
print("finalized")
#optimize_threshold(dt)
#print("optimized")
save_model(dt, "e:/rebid/amex/models/dt1Lclassfin")

In [ ]:
calibrated_dt = calibrate_model(dt)
save_model(calibrated_dt, "/amex/models/dt1Lclasscalib")

In [ ]:
plot_model(dt, plot='calibration')
plot_model(model, plot='calibration')

In [ ]:
dtmodels=[]
dtsets=[]
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns
n=6
algos=['dt', 'catboost', 'lightgbm']
loadconfig=True
loadmodels=[False, False, False]

for i in range(1):
    if loadconfig:
        dtsets.append(load_config(root+"/configs/" + 'dt' + str(n) + "-" + str(i) + "config.pkl"))
    else:
        print(i, end="")
        traindf=pd.DataFrame()
        for j in range(i*n, (i*n)+n):
            print("."+str(j), end="")
            traindf = pd.concat([traindf, pd.read_csv(trains[i]+"new1.csv", names=headers, header=None)], ignore_index=True)

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        dtsets.append(setup(data=traindf, target='target',
              transformation=True,
              ignore_features=['customer_ID'],
              categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
              numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
              remove_outliers=True, handle_unknown_categorical=True,
              combine_rare_levels=True, 
              create_clusters=True,
#              transform_target=True, transform_target_method='yeo-johnson',
              polynomial_features=True,
              feature_selection=True, remove_multicollinearity=True, pca=True, ignore_low_variance=True,
              train_size=0.8, fold=6,
              use_gpu=True,
              silent = True, session_id = 123, verbose=True))
        save_config(root+"/configs/" + algo + str(n) + "-" + str(i) + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + str(n)+"-"+str(i)+"reg"))
        else:
            dtmodels.append(create_model(algos[j], fold=6, verbose=True))
            save_model(dtmodels[i], root+"/models/" + algos[j] + str(n)+"-"+str(i)+"reg")

dtmodels.append(stack_models(dtmodels, meta_model = dtmodels[len(dtmodels)-1]))
save_model(dtmodels[len(dtmodels)-1], root+"/models/" + "".join(algos) + "stackedreg")

In [ ]:
# 0 to 12 dates modeling
algos=['catboost']
loadconfig=True
loadmodels=[False, False, False, False]
d=4
nof=4
tag="new"
xfiles = ["x" + str((i*nof)+j) + "13dts.csv" for i in range(int(12/nof)) for j in range(nof)]

mdf=pd.DataFrame()
for m in range(0):
    print("x12-"+str(m)+"dtcnt.csv", end=" ")
    mdf = pd.concat([mdf, pd.read_csv("x12-"+str(m)+"dtcnt.csv")], ignore_index=True)

for i in range(1):
#    xfiles = ["x" + str((i*nof)+j) + "13dts.csv" for j in range(nof)]
    xfiles = ["x" + str(i+j) + "13dts.csv" for j in range(12-i)]
#    xfiles = ["x" + str(i) + "13dts.csv"]
    dtmodels=[]
    tag = "new" + str(i)
    if loadconfig:
        load_config(root+"/configs/" + "x13dts" + tag + "config.pkl")
    else:
        traindf=pd.DataFrame()
        for xfile in xfiles:
            print(xfile, end=" ")
            if xfiles.index(xfile) == 0:
                traindf = pd.concat([traindf, pd.read_csv(xfile)], ignore_index=True)
            else:
#                traindf = pd.concat([traindf, pd.read_csv(xfile)[traindf.columns]], ignore_index=True)
                traindf = pd.concat([traindf, pd.read_csv(xfile)], ignore_index=True)
#            print("train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())

#        if i > 4:
        print("orig train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())
#        traindf = pd.concat([traindf, mdf.loc[0:(i*5000),:][traindf.columns]], ignore_index=True)
#        print("train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or
                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        s = setup(data=traindf, target='target',
            transformation=True,
            normalize=True,
            ignore_features=['customer_ID'],
            categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or 
                                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)],
#            numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
            remove_outliers=True,
            handle_unknown_categorical=True,
            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
            trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=2, #d,
            feature_selection=True, feature_selection_threshold=0.9,
#            remove_multicollinearity=True, #pca=True,
            ignore_low_variance=True,
            train_size=0.9, fold=10,
            use_gpu=True,
            silent = True, session_id = 123, verbose=True)
        d -= 1
        save_config(root+"/configs/" + "x13dts" + tag + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + "-x13dts" + tag))
        else:
            try:
                dtmodels.append(create_model(algos[j], fold=10, verbose=True))
                save_model(dtmodels[j], root+"/models/" + algos[j] + "-x13dts" + tag)

                if algos[j] in ['dt', 'lightgbm']:
                    try:
                        dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=15, verbose=True)
                        save_model(dtmodels[j], root+"/models/" + algos[j] + "-x13dts" + tag)
                    except:
                        print("Unable to tune for: ", algos[j])
                        traceback.print_exc()
            except:
                print("Unable to build model for:", algos[j])
                traceback.print_exc()

In [ ]:
mdf=pd.read_csv("mdfdtcnt.csv")
pd.concat([mdf, traindf], ignore_index=True)


In [ ]:
dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=15, verbose=True)
save_model(dtmodels[j], root+"/models/" + algos[j] + "-x13dts" + tag)


In [ ]:
algos=['dt', 'catboost', 'lightgbm']
algos=['lightgbm']
loadconfig=True
loadmodels=[False, False, False, False]
d=6

for i in range(12):
    dtmodels=[]
    if loadconfig:
        load_config(root+"/configs/" + "x" + str(i) + "config.pkl")
    else    :
        print("x" + str(i) + "13dts.csv", end="")
        traindf = pd.read_csv("x" + str(i) + "13dts.csv")

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or
                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        s = setup(data=traindf, target='target',
            transformation=True,
            normalize=True,
            ignore_features=['customer_ID'],
            categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
            numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
            remove_outliers=True,
            handle_unknown_categorical=True,
            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
            trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=d,
            feature_selection=True, remove_multicollinearity=True, #pca=True,
            ignore_low_variance=True,
            train_size=0.8, fold=10,
            use_gpu=True,
            silent = True, session_id = 123, verbose=True)
        save_config(root+"/configs/" + "x" + str(i) + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + "-x"+str(i)+"reg"))
        else:
            try:
                dtmodels.append(create_model(algos[j], fold=10, verbose=True))
                save_model(dtmodels[j], root+"/models/" + algos[j] + "-x"+str(i)+"reg")

                if algos[j] in ['dt', 'lightgbm']:
                    try:
                        dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=30, verbose=True)
                        save_model(dtmodels[j], root+"/models/" + algos[j] + "-x"+str(i)+"reg")
                    except:
                        print("Unable to tune for: ", algos[j])
                        traceback.print_exc()
            except:
                print("Unable to build model for:", algos[j])
                traceback.print_exc()

##    stacked=stack_models(dtmodels, meta_model=dtmodels[len(dtmodels)-1])
##    save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    try:
#        stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
#        save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    except:
#        print("Unable to tune for stacking")
##    dtmodels.append(stacked)
    d -= 1 if ((i/3)==round((i/3))) else d


In [48]:
algos=['dt', 'catboost', 'lightgbm']
algos=['lightgbm', 'catboost']
loadconfig=[False, False, False, False, False]
loadmodel=[False, False, False, False, False]
tunemodel=[True, True, True, True, True]
d=2
dtmodels=[]
nof=4

for i in range(1):
    tag = "".join([str((i*nof)+j) for j in range(nof)])
    if loadconfig[i]:
        load_config(root+"/configs/" + "x12-" + tag + "config.pkl")
    else:
#        print("x12-" + str(i*nof) + "dtcnt.csv")
#        traindf = pd.read_csv("x12-" + str(i*nof) + "dtcnt.csv")
        traindf=pd.DataFrame()
        for j in range(nof):
#            if exists("x12-" + str((i*nof)+j) + "dtcnt.csv"):
            print("x12-" + str((i*nof)+j) + "dtcnt.csv", end=" ")
            traindf = pd.concat([traindf, pd.read_csv("x12-" + str((i*nof)+j) + "dtcnt.csv")], ignore_index=True)
#            else:
#                print("x12-" + str((i*nof)-1) + "dtcnt.csv")
#                traindf = pd.concat([traindf, pd.read_csv("x12-" + str((i*nof)-1) + "dtcnt.csv")], ignore_index=True)

#        traindf.to_csv("x12-"+str((i*3))+str((i*3)+1)+str((i*3)+2)+".csv")
#        print("wrote x12-"+str((i*3))+str((i*3)+1)+str((i*3)+2)+".csv")
        print("Loaded data...", "train count:", traindf.shape, traindf.groupby(['target'])['customer_ID'].count())

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or
                    ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)]:
#            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        s = setup(data=traindf, target='target',
            transformation=True,
            normalize=True,
            ignore_features=['customer_ID'],
            categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col) or ('day_of_month') in col or ('day_of_year' in col)],
#            numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
            remove_outliers=True,
            handle_unknown_categorical=True,
            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
#            trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=d,
            feature_selection=True, feature_selection_threshold=0.9,
#            remove_multicollinearity=True, #pca=True,
            ignore_low_variance=True,
            train_size=0.8, fold=10,
            use_gpu=True,
            silent = True, session_id = 123, verbose=True)
        traindf=pd.DataFrame() #to free up the memory
        save_config(root+"/configs/" + "x12-" + tag + "config.pkl")

    for j in range(len(algos)):
        if loadmodel[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + "-x12-"+tag+"reg"))
        else:
            try:
                dtmodels.append(create_model(algos[j], fold=10, verbose=True))
                save_model(dtmodels[j], root+"/models/" + algos[j] + "-x12-" + tag+"reg")
            except:
                print("Unable to build model for:", algos[j])
                traceback.print_exc()
        if tunemodel[j]:
            if algos[j] in ['dt', 'lightgbm']:
                try:
                    dtmodels[j] = tune_model(dtmodels[j], fold=10, search_library = "scikit-optimize", n_iter=15, verbose=True)
                    save_model(dtmodels[j], root+"/models/" + algos[j] + "-x12-"+ tag +"reg")
                except:
                    print("Unable to tune for: ", algos[j])
                    traceback.print_exc()

##stacked=stack_models(dtmodels, meta_model=dtmodels[len(dtmodels)-1])
##save_model(stacked, root+"/models/" + "x12-" + str(i) + "stackedreg")
#    try:
#        stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
#        save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    except:
#        print("Unable to tune for stacking")
##dtmodels.append(stacked)

x12-0dtcnt.csv x12-1dtcnt.csv x12-2dtcnt.csv x12-3dtcnt.csv Loaded data... train count: (80360, 2602) target
0    61621
1    18739
Name: customer_ID, dtype: int64


NameError: name 'setup' is not defined

In [ ]:
dtmodels[0] = tune_model(dtmodels[0], fold=10, search_library = "scikit-optimize", n_iter=50, verbose=True)


In [ ]:
traindf = pd.read_csv(root+"/xall.csv")
s = setup(data=traindf, target='target',
    transformation=True,
    normalize=True,
    ignore_features=['customer_ID'],
    categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
    numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
#            remove_outliers=True,
    handle_unknown_categorical=True,
#            combine_rare_levels=True,
#            create_clusters=True,
#            transform_target=True, transform_target_method='yeo-johnson',
#    trigonometry_features=True,
#            polynomial_features=True, polynomial_degree=d,
    feature_selection=True, remove_multicollinearity=True, #pca=True,
    ignore_low_variance=True,
    train_size=0.9, fold=10,
    use_gpu=True,
    silent = True, session_id = 123, verbose=True)


In [ ]:
model = create_model('lightgbm', verbose=True)
evaluate_model(model)

In [ ]:
tuned = tune_model(model, search_library = "scikit-optimize", verbose=True)

In [ ]:
evaluate_model(tuned)

In [ ]:
blend = blend_models(dtmodels)

In [ ]:
finmodel = finalize_model(blend)

In [ ]:
save_model(model, root+"/models/lightgbm-xall")
plot_model(model, 'error')

In [ ]:
save_model(tuned, root+"/models/tuned-lightgbm-xall")
plot_model(tuned, 'error')

In [ ]:
dtmodels=[]
dtsets=[]
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns
n=18
algos=['dt', 'catboost', 'lightgbm']

for i in range(1):
#    for algo in algos:
#        if algo == "catboost":
#            dtmodels.append(load_model(root+"/models/" + algo + str(n)+"-"+str(i)+"reg"))
#    dtmodels.append(load_model(root+"/models/" + "".join(algos) + str(n) + "-" + str(i) + "stackedreg"))
    dtmodels.append(load_model(root+"/models/" + "x" + str(i) + "stackedreg"))

In [ ]:
stacked=stack_models(dtmodels, meta_model=dtmodels[0])
#stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
dtmodels.append(stacked)
save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")

In [ ]:
tests = []
for r, d, f in os.walk(root+"test"):
#    print(r)
#    if r ==root+'/test':
        f.sort()
        for file in f:
            if '.idx' not in file and '.csv' not in file and 'cust' not in file and 'preds' not in file and ('testa' in file or 'testb' in file or 'testc' in file or 'testd' in file or 'teste' in file or 'testf' in file):
                tests.append(os.path.join(r, file))
len(tests)#, tests[len(tests)-1]

In [ ]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

testdf = pd.read_csv(root+"/tstdata")
tdf = testdf.groupby('customer_ID')['S_2'].agg(['min', 'max', 'count']).reset_index()
for i in range(1, 14):
    print(i, tdf[tdf['count']==i]['count'].count())

tdf.rename(columns={"min": "mindate", "max": "maxdate"}, inplace=True)

tdf['maxdate'] = pd.to_datetime(tdf['maxdate'])
tdf['mindate'] = pd.to_datetime(tdf['mindate'])

#tdf['diffcount'] = tdf.apply(lambda x: ((x['maxdate'] - x['mindate'])/np.timedelta64(1, 'M')))

tdf['mondiff'] = (tdf.maxdate.dt.strftime("%Y%m").astype(int) - tdf.mindate.dt.strftime("%Y%m").astype(int) - 87)
tdf['mondiff'] = tdf['mondiff'].apply(lambda x: x+88 if x< 0 else x)
tdf[tdf['count'] != tdf['mondiff']]

In [ ]:
testdfs = []

for k in range(13):
    testdfs.append(pd.DataFrame(columns=headers))

k=0
for i in range(len(tests)):
    print(tests[i])
    testdf = pd.read_csv(tests[i], names=headers, header=None)
    for j in range(1, 14):
        mask = testdf.customer_ID.isin(tdf[tdf['mondiff'] == j].customer_ID)
        testdfs[j-1] = pd.concat([testdfs[j-1], testdf[mask]], ignore_index=True)
        if testdfs[j-1]['customer_ID'].count() >= 260000:
            testdfs[j-1].iloc[:260000,].to_csv("t" + str(j-1) + "-" + str(k) + ".csv", index=False)
            print("t" + str(i) + "-" + str(k) + ".csv")
            testdfs[j-1] = testdfs[j-1].iloc[260000:,]
            k+=1

for i in range(12):
    testdfs[i].to_csv("t" + str(i) + ".csv", index=False)
    print("t" + str(i) + ".csv")

for i in range(12, len(testdfs)):
    testdfs[i].to_csv("t" + str(i) + "-" + str(k) + ".csv", index=False)
    print("t" + str(i) + "-" + str(k) + ".csv")


In [ ]:
testdfs = []

for k in range(13):
    testdfs.append(pd.DataFrame(columns=headers))
k=0
for i in range(len(tests)):
    print(tests[i])
    testdf = pd.read_csv(tests[i], names=headers, header=None)
    for j in range(13, 14):
        mask = testdf.customer_ID.isin(tdf[tdf['S_2']==j].customer_ID)
        testdfs[j-1] = pd.concat([testdfs[j-1], testdf[mask]], ignore_index=True)
        if testdfs[j-1]['customer_ID'].count() >= 260000:
            testdfs[j-1].iloc[:260000,].to_csv("t" + str(j-1) + "-" + str(k) + ".csv", index=False)
            testdfs[j-1] = testdfs[j-1].iloc[260000:,]
            k+=1            

testdfs[12].to_csv("t12-" + str(k) + ".csv", index=False)


In [ ]:
predictions=pd.DataFrame()
for i in range(11, 12): #, 12):
    model = dtmodels[len(dtmodels)-1] #load_model(root+"/models/" + "x" + str(i) + "stackedreg")
    print("t" + str(i) + "dtcnt.csv")
    testdf = (pd.read_csv("t" + str(i) + "dtcnt.csv"))

    preds = predict_model(model, data=testdf, verbose=True)
    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
    preds[['customer_ID', 'Label']].to_csv(root+"/test/t"+ str(i) + ".dtcnt" + ".preds", header=False, index=False)

#predictions.to_csv("test.preds", header=False, index=False)

In [ ]:
testdfs = []
i=0
#mdf = pd.DataFrame()
#for m in range(2):
#    mdf = pd.concat([mdf, pd.read_csv("t12-"+str(m)+".csv")], ignore_index=True)
#mdf['tag'] = 12
#mdf['S_2'] = pd.to_datetime(mdf['S_2'])
for j in range(12):
    print("t" + str(j), end=" ")
    testdfs.append(pd.read_csv("t" + str(j) + ".csv"))

    maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
    mindate = maxdate + pd.offsets.DateOffset(months= -1-j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=dates['seq']+1

    testdfs[i]['S_2'] = pd.to_datetime(testdfs[i]['S_2'])
    testdfs[i]['yearmonthorig'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
    testdfs[i]['seq'] = testdfs[i].groupby(['customer_ID']).cumcount().add(1)
    testdfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = testdfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.strftime("%Y%m").astype(int) - (maxdate.year*100 + maxdate.month)

#    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2019 and x['mondiff'] > j else x['mondiff'], axis=1)
#    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <=0 else x)

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(years=1) if x['S_2'].year==2019 and x['S_2'].month > 4 else x['S_2'], axis=1)
    testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")

#    testdfs[i]['ncount'] = testdfs[i].isnull().sum(axis=1)

    testdfs[i]=pd.merge(testdfs[i], pd.merge(pd.DataFrame(testdfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
#    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
    testdfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
#    testdfs[i] = testdfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    testdfs[i]['yearmonthorig'].fillna(0, inplace=True)
#    testdfs[i]['mondiff'].fillna(0, inplace=True)
    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['mondiff'], inplace=True)
#    testdfs[i].drop(columns=['S_2'], inplace=True)

#    testdfs[i]['tag'] = 0
#    testdfs[i] = pd.concat([testdfs[i], mdf], ignore_index=True)

#    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
#    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
#    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
#    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['S_2'], inplace=True)

#    cols=testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#    testdfs[i].drop(testdfs[i][testdfs[i]['tag']==12].index, axis=0, inplace=True)
#    testdfs[i].drop(columns=['tag'], inplace=True)

    testdfs[i]=testdfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    testdfs[i] = testdfs[i].reset_index()

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    testdfs[i].fillna(0, inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][[column for column in testdfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][col].mean(), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(-999, inplace=True)

    testdfs[i].drop(columns=[testdfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in testdfs[i].columns:
        coldict[col] = ''.join(col)
    testdfs[i].rename(coldict, axis=1, inplace=True)

    testdfs[i].to_csv("t" + str(j) +"13dts.csv", index=False)
    i+=1

testdfs[0]


In [ ]:
mdf=pd.read_csv("mdfdtcnt.csv")
for j in range(12):
    print(j, end=" ")
    pd.concat([mdf, traindfs[j]], ignore_index=True).to_csv("x" + str(j) + "13dts.csv", index=False)
    pd.concat([mdf[mdf.columns.tolist()[:len(mdf.columns)-2]], testdfs[j]], ignore_index=True).to_csv("t" + str(j) + "13dts.csv", index=False)


In [19]:
testdfs = []
i=0
maxdate = datetime.strptime('Apr 30 2019 1:33PM', '%b %d %Y %I:%M%p')
mindate = maxdate + pd.offsets.DateOffset(months= -13)
dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
dates.reset_index(inplace=True)
dates.rename(columns={'index': 'seq'}, inplace=True)
dates['seq']=dates['seq']+1

for j in range(6):
    print("t12-" + str(j), end=" ")
    testdfs.append(pd.read_csv("t12-" + str(j) + ".csv"))

    testdfs[i]['S_2'] = pd.to_datetime(testdfs[i]['S_2'])
    testdfs[i]['yearmonthorig'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
    testdfs[i]['seq'] = testdfs[i].groupby(['customer_ID']).cumcount().add(1)
    testdfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = testdfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.strftime("%Y%m").astype(int) - (maxdate.year*100 + maxdate.month)

#    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
#    dateshift['mondiff'] = dateshift.apply(lambda x: 12+x['mondiff'] if x['S_2'].year == 2019 and x['mondiff'] > j else x['mondiff'], axis=1)
#    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <=0 else x)

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(years=1) if x['S_2'].year==2019 and x['S_2'].month > 4 else x['S_2'], axis=1)
    testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")

#    testdfs[i]['ncount'] = testdfs[i].isnull().sum(axis=1)

    testdfs[i]=pd.merge(testdfs[i], pd.merge(pd.DataFrame(testdfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
#    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
#    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: (2**(x['seq_y']-1)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    testdfs[i]['origmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2']) and x['seq_y'] <=j) else 0, axis=1)
    testdfs[i]['totmissrcnt'] = testdfs[i].apply(lambda x: 1 if (pd.isnull(x['S_2'])) else 0, axis=1)
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    dateshift = pd.merge(dateshift, pd.DataFrame(testdfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
    testdfs[i].drop(columns=['S_2_dt', 'seq_x', 'seq_y'], inplace=True)
#    testdfs[i] = testdfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    testdfs[i]['yearmonthorig'].fillna(0, inplace=True)
#    testdfs[i]['mondiff'].fillna(0, inplace=True)
    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['mondiff'], inplace=True)
#    testdfs[i].drop(columns=['S_2'], inplace=True)

#    testdfs[i]['tag'] = 0
#    testdfs[i] = pd.concat([testdfs[i], mdf], ignore_index=True)

#    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
#    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
#    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
#    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['S_2'], inplace=True)

#    cols=testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index
#    cols = [col for col in cols if col[0:1] in ['S','R', 'B', 'D', 'P']]
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_year'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['day_of_month'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth', 'day_of_week'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))
#    testdfs[i][cols] = testdfs[i].groupby(['yearmonth'])[cols].transform(lambda x : x.fillna(x.iloc[np.random.choice(range(0,len(x)))]))

#    testdfs[i][cols] = testdfs[i].groupby(['day_of_week'])[cols].transform(lambda x: x.fillna(x.mean()))

#    testdfs[i].drop(testdfs[i][testdfs[i]['tag']==12].index, axis=0, inplace=True)
#    testdfs[i].drop(columns=['tag'], inplace=True)

    testdfs[i]=testdfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    testdfs[i] = testdfs[i].reset_index()

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

#    testdfs[i].fillna(0, inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][[column for column in testdfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][col].mean(), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(-999, inplace=True)

    testdfs[i].drop(columns=[testdfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in testdfs[i].columns:
        coldict[col] = ''.join(col)
    testdfs[i].rename(coldict, axis=1, inplace=True)

    testdfs[i].to_csv("t12-" + str(j) +"dtcnt.csv", index=False)
#    os.remove(tests[i]
    i+=1

testdfs[0]

t12-0.csv
t12-1.csv
t12-2.csv
t12-3.csv
t12-4.csv
t12-5.csv


,customer_ID,B_1201804,B_10201804,B_11201804,B_12201804,B_13201804,B_14201804,B_15201804,B_16201804,B_17201804,...,dncnt201904,origmissrcnt201904,pncnt201904,rncnt201904,sncnt201904,totmissrcnt201904,week_of_year201904,yearmonthorig201904,origmissrcnt,totmissrcnt
0,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.020970,0.295178,0.010088,0.024187,0.030294,0.009963,0.005224,0.009955,NaN,...,1156.0,0,0.0,37.0,0.0,0,16,201904,0,0
1,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.031007,0.298906,0.021077,0.055798,0.075478,0.039006,0.045434,0.001200,NaN,...,957.0,0,0.0,37.0,0.0,0,16,201910,0,0
2,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.269828,0.013830,0.276393,0.079677,0.071900,0.418853,0.319683,1.007010,1.007430,...,1017.0,0,0.0,37.0,0.0,0,17,201904,0,0
3,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.381377,0.031143,0.347868,0.013177,0.037534,0.140309,0.001664,1.006746,1.001244,...,1198.0,0,0.0,10.0,0.0,0,17,201910,0,0
4,00004ffe6e01e1b688170bbd108da8351bc4c316eacfef48643ee028ab947afc,0.011573,0.298053,0.003080,0.009443,0.006702,0.011710,0.008398,0.086959,NaN,...,1147.0,0,0.0,37.0,0.0,0,14,201910,0,0
5,00007cfcce97abfa0b4fa0647986157281d01d3ab90de919398cc0ba335710b5,0.233648,0.028343,0.198970,0.011453,0.021303,0.080147,0.009390,0.837666,1.006792,...,930.0,0,0.0,10.0,0.0,0,17,201904,0,0
6,000089cc2a30dad8e6ba39126f9d86df6088c9f975093a2e80b555b72b232406,0.873518,0.014569,0.834611,0.010035,0.040350,0.200822,0.006819,1.002766,1.003489,...,1053.0,0,0.0,10.0,0.0,0,17,201904,0,0
7,00008f50a1dd76fa211ba36a2b0d5a1b201e4134a5fd53d04f1693d4abb70b2b,0.300863,0.008047,0.255422,0.008742,0.010463,0.072932,0.005175,1.005592,0.968347,...,1138.0,0,0.0,10.0,0.0,0,17,201910,0,0
8,0000b48a4f27dc1d61e78d081678e811620300b88eb3ab25d3234e2a2a3c9200,0.023918,0.295226,0.009627,0.028225,0.016608,0.013589,0.016031,0.002202,NaN,...,1170.0,0,0.0,37.0,36.0,0,15,201910,0,0
9,0000bccc55cf039a23d49832234f224085716d85bed7d2b83316c835ed0a0c1b,0.053669,0.016973,0.027354,0.011527,0.005171,0.014302,0.006330,1.002615,0.653189,...,1071.0,0,0.0,10.0,27.0,0,15,201904,0,0


In [ ]:
testdfs[0]
#dateshift.sort_values('mondiff', ascending=False)
#dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()
#dateshift['min'] = pd.to_datetime(dateshift['min'])
#dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])

#dateshift['mondiff'] = dateshift['S_2'].dt.strftime("%Y%m").astype(int) - dateshift['min'].dt.strftime("%Y%m").astype(int) - 87
#dateshift[dateshift['customer_ID'] == '064b1f0072facb6599248737d0b1c9f011eba59233794057beb66b1dc13d647a']

testdfs[0][testdfs[i]['customer_ID'] == '064b1f0072facb6599248737d0b1c9f011eba59233794057beb66b1dc13d647a']

#pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'max', 'count'])).rename(columns={"max": "S_2"}).reset_index()

testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
tst = testdfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1).reset_index()
#testdfs[i][testdfs[i]['customer_ID'] == '064aee43581981bdcf7e2906f8cb875a503445ed7ac2d40ad68a03c213398e79']
#tst[tst['customer_ID'] == '064aee43581981bdcf7e2906f8cb875a503445ed7ac2d40ad68a03c213398e79']

dateshift

dateshift['S_2'].dt.strftime("%Y%m").astype(int) - (maxdate.year*100 + maxdate.month) 

In [ ]:
#model = load_model("/amex/models/dt1Lclass")
#model = calibrated_dt
#model=stacked
#model=load_model("/amex/models/dtstackedreg")

useFlag = False
mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

predictions=pd.DataFrame()
for i in range(104, len(tests)):
    print(tests[i].split("/")[5], end =" ")
    if (not useFlag):
        headers=pd.read_csv(root+"headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i], names=headers, header=None)
        test['S_2'] = pd.to_datetime(test['S_2'])

        dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
        dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
        dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

        test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
        test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
        test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

        test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
        test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)

        test['day_of_month'] = test['S_2'].dt.strftime("%d")
        test['day_of_week'] = test['S_2'].dt.strftime("%u")
        test['day_of_year'] = test['S_2'].dt.strftime("%j")
        test.drop(columns=['S_2_dt'], inplace=True)
        test.drop(columns=['mondiff'], inplace=True)

        test = test.set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
        test = test.reset_index()
        test = pd.merge(test, dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

        test = test.merge(labels, on="customer_ID", how="left")
        test.drop(columns=[test.columns.to_list()[1]], inplace=True)

        coldict={}
        for col in test.columns:
            coldict[col] = ''.join(col)

        test.rename(coldict, axis=1, inplace=True)
        test['target'] = 0

#        test.to_csv(tests[i]+".csv", index=False, header=False)
    else:
        headers=pd.read_csv(root+"/trains/headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i]+".csv", names=headers, header=None)

    for col in [col for col in test.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
        test[col] = test[col].astype(str)
        test[col] = test[col].astype('category')

    test['customer_ID2'] = test['customer_ID']
    test.drop(columns=['customer_ID'], inplace=True)
    test.rename(columns = {"customer_ID2": "customer_ID"}, inplace=True)
    try:
        preds = predict_model(dtmodels[len(dtmodels)-1], data=test, verbose=True)
    except:
        preds = predict_model(dtmodels[0], data=test, verbose=True)
    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
#    preds[['customer_ID', 'Label']].to_csv(tests[i]+"." + "".join(algos) + s
    preds[['customer_ID', 'Label']].to_csv(tests[i]+".dt2" + ".preds", header=False, index=False)

predictions.to_csv("test.preds", header=False, index=False)

In [ ]:
from datetime import datetime

mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

test = pd.read_csv(tests[i], names=headers, header=None)
for col in [col for col in test.columns if col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]:
    test[col] = test[col].astype(str)
    test[col] = test[col].astype('category')

test['S_2'] = pd.to_datetime(test['S_2'])

dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].min()).reset_index()
dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)


In [ ]:
print((test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*12 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month) - (mindate.year*12+mindate.month), (test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*100 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month), (mindate.year*100+mindate.month))
test
test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']

In [ ]:
def testcolfillna(col):
#    print(col, end="")
    try:
        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]
    except:
        print(col)
    return

def testfillna(col):
#    print(".", end="")
    test[col].fillna(test[[column for column in test.columns.tolist() if column[:len(column)-3] in col]].mean(axis=1), inplace=True)

#    with Pool(12) as pool:
#        pool.map(testcolfillna, [col for col in test.columns[test.isna().all()]])

#    with Pool(12) as pool:
#        pool.map(testfillna, [col for col in test.columns[test.isna().any()]])

#    for col in test.columns[test.isna().all()]:
#        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]



In [ ]:
traindf = pd.read_csv(root+"/x12-0.csv")
traindf

In [ ]:
df=pd.read_csv(root+"/x3dtcnt.csv")
df[df['yearmonthorig201903'] == 0]